# 22.5 Strongly connected components

In a strongly connected componentn (SCC) of a directed graph $G=(V,E)$, **every** pair of vertices $(u,v)$ are **reachable** from each other. That is, if $C$ and $C'$ are **distinct** SCC, such pair $(u,v)$ does not exist. 

`scc` is a linear-time (i.e. $\Theta (V+E)$-time) algorithm computes the strongly connected components of a directed graph $G=(V,E)$ using two depth-first search (DFS) on $G$ and its tranpose $G^T$:

1. By DFS on $G$, topologically sort the vertices of $G$
2. Compute $G^T$
3. DFS on $G^T$ with iteration of vertices in the order of 1)

The DFS trees obtained from 3) are the distinct strongly connected components.

`scc` depends on three helper functions we defined previously:
* `transponse_list` from [22.1-3](22.1_Representations_of_graphs_Exercise.ipynb) to transponse an adjacency list
* `dfs_ntree` from [22.3-12](22.3_Depth-first_search_Exercise.ipynb) to compute dfs tree from a depth-first search
* `topological_sort` from [22.3-4](22.4_Topological_sort_Exercise.ipynb) to topologically sort a given adjacency list

In [1]:
def transpose_list(G):
    # Gt is an empty list
    Gt={i:[] for i in G.keys()}
    # scan through G
    for u in G:
        for v in G[u]:
            Gt[v].append(u)
    return Gt
def dfs_ntree(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    global ntree #number of connected component
    ntree=0
    cc={} #attribute u.cc
    for u in G:
        if colour[u]=='white':
            ntree+=1 # increment ntree for a start of a new dfs tree
            dfs_visit_ntree(G, u, colour, t_discover,t_finish,predecessor,cc) #recursion
    return cc#, colour, t_discover, t_finish, predecessor 

def dfs_visit_ntree(G, u, colour, t_discover,t_finish,predecessor,cc):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    cc[u]=ntree # for every other tree inside the dfs, they have the same ntree
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_ntree(G, v, colour, t_discover,t_finish,predecessor,cc)
    colour[u]='black'
    time+=1
    t_finish[u]=time  
    
from collections import deque
def topological_sort(G):
    global time # time is a global varaible, it is accumulative
    time=0
    colour={i:'white' for i in G} # initial colour of all vertices are white
    t_discover={}
    t_finish={}
    predecessor={}
    sorted_list=deque() # to store the sorted list
    for u in G:
        if colour[u]=='white':
            dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list) #recursion
    return sorted_list #,t_discover, t_finish

def dfs_visit_topo(G, u, colour, t_discover,t_finish,predecessor,sorted_list):
    global time
    time+=1
    t_discover[u]=time # a white vertex is discovered
    colour[u]='grey' # and coloured to grey
    for v in G[u]:
        if colour[v]=='white':
            predecessor[v]=u
            dfs_visit_topo(G, v, colour, t_discover,t_finish,predecessor,sorted_list)
    sorted_list.appendleft(u) # a vertex fully explored is appended on the left of the list
    colour[u]='black'
    time+=1
    t_finish[u]=time 

In [2]:
def scc(G):
    # compute tranpose of G
    Gt=transpose_list(G)
    
    # topological sorted vertices of G
    sorted_vertices=topological_sort(G)
    
    # rearrange Gt according to topologically sorted vertices in G
    # this is done by merging GG, an empty dict "sorted" according to sorted_vertices
    # and Gt
    GG={i:[] for i in sorted_vertices}
    Gt={**GG, **Gt}
   
    # display dfs trees of Gt in order of sorted_vertices
    return dfs_ntree(Gt)

Using **Figure 22.9** as an example, running `scc` gives 4 distinct SCC.
<img src="img/fig22.9a.png" width="700">

In [3]:
G1={'a':['b'],
    'b':['c','e','f'],
    'c':['d','g'],
    'd':['c','h'],
    'e':['a','f'],
    'f':['g'],
    'g':['f','h'],
    'h':['h']}   
scc(G1)

{'a': 1, 'e': 1, 'b': 1, 'c': 2, 'd': 2, 'g': 3, 'f': 3, 'h': 4}